### Example of using CMSSpark for analysis workflow

In [1]:
# IMPORTANT!!!
# ------------
# Before running this notebook please click on star icon (if it is there) to connect to Spark cluster
# it setups spark context variables used in this notebook

In [2]:
# execute this cell which downloads required libraries and put them in place
import os
with open('env.sh', 'w') as ostream:
    ostream.write("""#!/bin/bash
wdir=$PWD/wdir
if [ -d $wdir ]; then
    rm -rf $wdir
fi
mkdir -p $wdir
cd $wdir
wget https://my.pcloud.com/publink/show?code=XZwzn97ZIoG4isDNO3b6SrTEUfQ1Y4E6bp9k -O ./avro-mapred.jar
wget https://my.pcloud.com/publink/show?code=XZazn97Z9aGq81aVgBLcHi9LBNqryuqs310V -O ./spark-csv-assembly-1.4.0.jar
wget http://cmsdoc.cern.ch/~dciangot/spark-examples.jar -O ./spark-examples-1.6.0-cdh5.7.6-hadoop2.6.0-cdh5.7.6.jar
wget https://github.com/dmwm/CMSSpark/archive/master.zip -O ./master.zip
unzip ./master.zip 2> /dev/null
mv CMSSpark-master CMSSpark""")
!ls
!source ./env.sh
!ls
!cd wdir; ls

analyzer.ipynb		panduplizer.ipynb	    ws_17_05.pkl
analyzer-MINI.ipynb	plot.pdf		    ws_17_06.pkl
analyzer-noMINI.ipynb	plots			    ws_17_07.pkl
cache_emulator.ipynb	spark-dfs		    ws_17_08.pkl
condor_20180101		spark-warehouse		    ws_17_09.pkl
dataset_producer.ipynb	wdir			    ws_17_10.pkl
env.sh			working_set_classads.ipynb  ws_17_11.pkl
mkdir.sh		ws_17_01.pkl		    ws_17_12.pkl
name			ws_17_02.pkl		    xdf_raw_jan_2018.pkl.gz
pandas_dfs		ws_17_03.pkl
pandas_raw		ws_17_04.pkl
--2019-05-12 18:27:06--  https://my.pcloud.com/publink/show?code=XZwzn97ZIoG4isDNO3b6SrTEUfQ1Y4E6bp9k
Resolving my.pcloud.com (my.pcloud.com)... 74.120.8.7, 74.120.9.89, 74.120.8.6, ...
Connecting to my.pcloud.com (my.pcloud.com)|74.120.8.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43263 (42K) [text/html]
Saving to: ‘./avro-mapred.jar’

100%[======================================>] 43,263      --.-K/s   in 0.1s    

2019-05-12 18:27:06 (321 KB/s) - ‘./avro-mapred.jar’ saved [43263/

analyzer.ipynb		panduplizer.ipynb	    ws_17_05.pkl
analyzer-MINI.ipynb	plot.pdf		    ws_17_06.pkl
analyzer-noMINI.ipynb	plots			    ws_17_07.pkl
cache_emulator.ipynb	spark-dfs		    ws_17_08.pkl
condor_20180101		spark-warehouse		    ws_17_09.pkl
dataset_producer.ipynb	wdir			    ws_17_10.pkl
env.sh			working_set_classads.ipynb  ws_17_11.pkl
mkdir.sh		ws_17_01.pkl		    ws_17_12.pkl
name			ws_17_02.pkl		    xdf_raw_jan_2018.pkl.gz
pandas_dfs		ws_17_03.pkl
pandas_raw		ws_17_04.pkl
avro-mapred.jar  spark-csv-assembly-1.4.0.jar
CMSSpark	 spark-examples-1.6.0-cdh5.7.6-hadoop2.6.0-cdh5.7.6.jar
master.zip


In [3]:
# include the CMSSpark python in the sys.path
import os,sys
sys.path.append(os.path.join(os.path.expanduser("~"), 'SWAN_projects/CMSSpark/src/python'))
#print(sys.path)


In [4]:
#!ls ws_classAds_csv_mcTrue_19_01/part-*.csv.gz
#print glob.glob(os.path.join('', "ws_classAds_csv_mcTrue_19_%s/part-*.csv.gz"))
!hdfs dfs -ls  hdfs://analytix/cms/users/dciangot

Found 32 items
drwxr-xr-x   - dciangot zh          0 2019-05-10 23:02 hdfs://analytix/cms/users/dciangot/block_size
drwxr-xr-x   - dciangot zh          0 2019-05-11 20:15 hdfs://analytix/cms/users/dciangot/ws_classAds_mcFalse_18_01
drwxr-xr-x   - dciangot zh          0 2019-05-11 21:26 hdfs://analytix/cms/users/dciangot/ws_classAds_mcFalse_18_02
drwxr-xr-x   - dciangot zh          0 2019-05-12 01:14 hdfs://analytix/cms/users/dciangot/ws_classAds_mcFalse_18_03
drwxr-xr-x   - dciangot zh          0 2019-05-12 02:36 hdfs://analytix/cms/users/dciangot/ws_classAds_mcFalse_18_04
drwxr-xr-x   - dciangot zh          0 2019-05-12 04:33 hdfs://analytix/cms/users/dciangot/ws_classAds_mcFalse_18_05
drwxr-xr-x   - dciangot zh          0 2019-05-12 05:42 hdfs://analytix/cms/users/dciangot/ws_classAds_mcFalse_18_06
drwxr-xr-x   - dciangot zh          0 2019-05-12 06:22 hdfs://analytix/cms/users/dciangot/ws_classAds_mcFalse_18_07
drwxr-xr-x   - dciangot zh          0 2019-05-12 07:25 hdfs://analytix/c

In [7]:
#!hdfs dfs -cp  hdfs://analytix/cms/users/dciangot/ws_classAds_mc*_18_{09,10,11,12} file:${PWD}/spark-dfs/
!hdfs dfs -cp  hdfs://analytix/cms/users/dciangot/ws_classAds_mc*_18_{12} file:${PWD}/spark-dfs/

In [9]:
##working_set_day.head(10)
from pyspark.sql import SQLContext
import subprocess
import pandas as pd

sqlContext = SQLContext(sc)

mcList = ['False', 'True']
#list = ['18_01', '18_02', '18_03', '18_04', '18_05', '18_06', '18_07', '18_08', '18_09', '18_10', '18_11', '18_12']
list = ['18_12']
#list = ['19_02', '19_03', '19_04']

#!pip install --user -U pyarrow

#import sys
#sys.path.append('/usr/lib/python2.7/site-packages')

for mc in mcList:
    for i in list:
        print mc, i

        import glob, os
        all_files = glob.glob(os.path.join('', "./spark-dfs/ws_classAds_mc%s_%s/part-*" % (mc, i)))

        li = []    
        j = 0
        for filename in all_files:
            df = pd.read_parquet(filename)
            li.append(df)

        df = pd.concat(li, axis=0)
        #    print "Saving...", j
        df.to_pickle("./pandas_raw/ws_mc%s_%s.pkl" % (mc, i))

    

False 18_12
True 18_12
